In [3]:
from threading import Thread,Event
import threading
import time,random


def conn_mysql():
    count=1
    while not event.is_set():
        if count > 3:
            raise TimeoutError('链接超时')
        print('<%s>第%s次尝试链接' % (threading.current_thread().getName(), count))
        event.wait(0.5)
        count+=1
    print('<%s>链接成功' %threading.current_thread().getName())
    

def check_mysql():
    print('\033[45m[%s]正在检查mysql\033[0m' % threading.current_thread().getName())
    time.sleep(random.randint(2,4))
    event.set()
    
    
if __name__ == '__main__':
    event=Event()
    conn1=Thread(target=conn_mysql)
    conn2=Thread(target=conn_mysql)
    check=Thread(target=check_mysql)

    conn1.start()
    conn2.start()
    check.start()

<Thread-4>第1次尝试链接
<Thread-5>第1次尝试链接
[Thread-6]正在检查mysql
<Thread-4>第2次尝试链接
<Thread-5>第2次尝试链接
<Thread-4>第3次尝试链接
<Thread-5>第3次尝试链接


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/sun/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/sun/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-dd3bc5b95882>", line 8, in conn_mysql
    raise TimeoutError('链接超时')
TimeoutError: 链接超时

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/sun/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/sun/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-dd3bc5b95882>", line 8, in conn_mysql
    raise TimeoutError('链接超时')
TimeoutError: 链接超时



In [36]:
import threading
n = 0
def foo():
    global n
    n += 1
threads = []
for i in range(100):
    t = threading.Thread(target=foo)
    threads.append(t)
for t in threads:
    t.start()
for t in threads:
    t.join()
print(n)

100


# 协程
- 参考资料

# 迭代器
- 可迭代(Iterable):直接作用于for循环的变量
- 迭代器(Iterable):不但可以作用于for循环, 还可以被next调用.
- list是典型的可迭代对象, 但不是迭代器
- 通过isinstance进行判断
- 通过iter()进行转换

In [37]:
# 可迭代
l = [i for i in range(10)]
print(l)

# range是个迭代器
for i in range(5):
    print(i)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
0
1
2
3
4


In [39]:
# 判断是否可迭代

from collections import Iterable
from collections import Iterator

l1 = [1,2,3,4,5]

print(isinstance(l1, Iterable))
print(isinstance(l1, Iterator))

True
False


In [47]:
# iter()转换
s = 'asdfkha;孙宇军gds阿斯顿jjk'

print(isinstance(s, Iterable))
print(isinstance(s, Iterator))

s_iter = iter(s)

print(isinstance(s_iter, Iterable))
print(isinstance(s_iter, Iterator))

# 那么迭代器的话...就能next了吧↓
    
try:
    while True:
        print(next(s_iter), end='.')
except StopIteration:
    print("\n完事儿里奥")

True
False
True
True
a.s.d.f.k.h.a.;.孙.宇.军.g.d.s.阿.斯.顿.j.j.k.
完事儿里奥


# 生成器(generator)
- 一边循环, 一边计算下一个元素的机制/算法
- 需要满足三个条件:
    - 每次调用都生成for循环需要的下一个元素
    - 如果打到最后一个, 爆出StopIteration异常
    - 可以被next()函数调用
- 如何制作一个生成器
    - 直接使用
    - 如果函数中包含yield, 则这个函数就叫生成器
    - next调用函数, 遇到yield会返回

In [54]:
# 直接使用生成器

L = [x*x for x in range(5)]  # 放在中括号就是列表生成器

g = (x*x for x in range(5))  # 放在小括号就是生成器

print(type(L))
print(type(g))


<class 'list'>
<class 'generator'>


In [55]:
# 函数案例

def odd():
    print('Step 1')
    print('Step 2')
    print('Step 3')
    return None

a = odd()

Step 1
Step 2
Step 3


In [58]:
# 生成器案例
# 在odd函数中, yield负责返回
def odd():
    print('Step 1')
    yield 1
    print('Step 2')
    yield 2
    print('Step 3')
    yield 3

# odd()是调用生成器, 这里普通函数会打印'Step 1',然而此处并没有, 因为里面有yield, 是生成器.
g = odd()
print("已经调用了")

one = next(g) # 在调用next之前, 生成器一动不动像王八, 调用之后, 会一直执行到yield, 并按本句代码的要求把yield后面
print(one)    # 的值赋给one然后就又一动不动像王八了. 

two = next(g)
print(two)

three = g.send('三')  # odd自会next, 无需再next
# try:
#     three = next(g)
# except StopIteration:
print(three)

已经调用了
Step 1
1
Step 2
2
Step 3
3


In [23]:
# for 循环调用生成器

def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        print(b)
        a, b = b, a+b
        n += 1
        
    return None

fib(5)

1
1
2
3
5


In [24]:
# feibonaqi数列生成器写法

def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        yield b
        a, b = b, a+b
        n += 1
        
    return None

g = fib(5)  # 记住了, 这里函数一动没动.

for i in range(6):
    rst = next(g)  # 看见next, 生成器被激活, 执行到yield后暂停, 把b的值赋给g, 注意, 是"暂停"!!! 
                   # 直到next再次出现, 再继续奔向下一个yield
    print(rst)

1
1
2
3
5


StopIteration: 

In [53]:
ge = fib(10)
"""
生成器的典型用法就是在for循环里使用
比较常用的典型生成器是range.
"""
# 我认为这里ge从一开始仍然是一动不动的, 直到for循环的出现, 它起到了next()的作用
for i in ge:
    print(i, end=',')


1,1,2,3,5,8,13,21,34,55,

# 协程

- 历史历程
    - 3.4引入协程, 用yield实现
    - 3.5引入协程语法
    - 实现协程比较好的包有: 
        - asyncio
        - tornado
        - gevent

- 定义: 协程, 十位非抢占式多任务产生子程序的计算机程序组件, 协程允许不同入口点在不同位置暂停或开始执行程续.

- 从技术角度讲, 协程就是一个你可以暂停执行的函数, 或者干脆把协程理解成生成器.

- 协程的实现:
    - yield调用
    - send调用
    
- 协程的四个状态
    - inspect.getgeneratorstate(...)函数确定, 该函数会返回下列字符串中的一个:
    - GEN_CREATED: 等待开始
    - GEN_RUNNIG: 解释器正在执行
    - GEN_SUSPENDED: 在yield处暂停
    - GEN_CLOSED: 执行结束
    - next预激(prime)

- 携程终止
    - StopIteration
    - 协程中未处理的异常会向上冒泡, 传给next函数或send方法的调用方 (即触发携程对象)
    - 停止协程的一种方式: 发送某个哨符值, 让协程退出. 内置的None和Ellipsis 等常量经常用作哨符值==

- yield from
    - 调用协程为了得到返回值, 协程必须正常终止.
    - 生成器正常终止会发出StopIteration异常.
    - yield from 从内部捕获StopIteration异常
    - 案例v03
    - 委派生成器
        - 包含yield from 表达式的生成器函数
        - 委派生成器在yield from表达式处暂停, 调用方可以直接把数据发给子生成器
        - 子生成器再把产生的值发给调用方
        - 子生成器在最后, 解释器会抛出StopIteration异常, 并且把返回值附加到异常对象上.
        - 案例04

In [61]:
# from functools import wraps

def coroutinue(func):
    '''
    装饰器： 向前执行到第一个`yield`表达式，预激`func`
    :param func: func name
    :return: primer
    '''    
#     @wraps(func)
    def primer(*args, **kwargs):
        # 把装饰器生成器函数替换成这里的primer函数；调用primer函数时，返回预激后的生成器。
        gen = func(*args, **kwargs)        # 调用被被装饰函数，获取生成器对象
        next(gen)  # 预激生成器
        return gen  # 返回生成器, 它正常情况是simple_coro的返回值, 但是由于有yield存在, 所以primer的返回值是一个生成器
    return primer

# 使用方法如下

@coroutinue
def simple_coro(a):
    print("预激",a)
    a = yield
    return a + 2
    
    
sc = simple_coro(12)  # 这句话实际上右边是 sc = primer(12), 而后者返回值是经过预激的simple_coro(12), 
sc.send(18)           # 而参数a的实参值对于结果而言并没有意义


预激 12


StopIteration: 20

In [9]:
import functools
help(functools.wraps)  # 网上示例所用， 不知道是什么意思

Help on function wraps in module functools:

wraps(wrapped, assigned=('__module__', '__name__', '__qualname__', '__doc__', '__annotations__'), updated=('__dict__',))
    Decorator factory to apply update_wrapper() to a wrapper function
    
    Returns a decorator that invokes update_wrapper() with the decorated
    function as the wrapper argument and the arguments to wraps() as the
    remaining arguments. Default arguments are as for update_wrapper().
    This is a convenience function to simplify applying partial() to
    update_wrapper().



In [65]:
# 协程代码案例1

def simple_coroutine():
    print('Started')
    x = yield
    print('Recived', x)

    
sc = simple_coroutine()

print(11111)
next(sc) # 预激

print(22222)
sc.send('啥玩意?')


11111
Started
22222
Recived 啥玩意?


StopIteration: 

In [74]:
def gen():
    for c in "AB":
        yield c
    return 3

g = gen()

print(next(g))
print(next(g))
print(next(g))

# 等价代码：(这也让我更好地理解了for循环的本质,)
'''
def gen():
    
    c = "A"
    yield c
    
    c = "B"
    yield c
    
    return 3
'''

A
B


StopIteration: 3

In [75]:
# v03

def gen():
    for c in "AB":
        yield c
# list直接用生成器作为参数， list就是调用方, 也是个会自动next的货色.
print(list(gen()))

def gen_new():
    yield from "AB"

print(list(gen_new()))

['A', 'B']
['A', 'B']


In [92]:
# 案例04, 委派生成器
from collections import namedtuple

ResClass = namedtuple('Res', 'count, average')

# 子生成器
def averager():
    total = 0.0
    count = 0
    average = None
    
    while True:
        if count == 0:
            print("3dada,对应每次预激")
        term = yield  # 第一次预激是到这里暂停的.
        
        if term is None:
            break
        total += term
        count += 1
        average = total / count

    return ResClass(count, average)

# 委派生成器
def grouper(storages, key):
    while True:  
        storages[key] = yield from averager()

# 调用方代码
def client():
    process_data = {
        'boys_1': [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
        'boys_2': [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46]
    }

    storages = {}
    for k, v in process_data.items():
        # 获得协程
        coroutine = grouper(storages, k)
        
        print('1dada,对应每个boy')
        # 预激协程, 注意这时候子生成器也已经执行到yield了. 因为在yield from处暂停时会把右侧的代码执行完
        next(coroutine)
        print("2dada,对应每次预激结束")
        
        # 发送数据到协程
        for dt in v:
            coroutine.send(dt)

        # 终止协程
        coroutine.send(None)
    print(storages)
'''
我们之所以看到'多余'的'3dada,是因为什么呢, 原来是委派生成器的while循环导致的, 它会在最后一次返回值的时候,又循环到yield from的
右侧,而此时count是等于0的,因为这次又从averager的最顶部开始跑到yield了.
那么索性把委派生成器的while去掉不行吗? 尝试发现, 去掉之后,StopIteration异常在boy_1结束后就冒泡到调用方,导致程序终止.这又是
为什么呢? debug走起.
我整明白了， 我太牛逼了， 我草了， 啊
- 过奖过奖， 具体看下个cell
'''
client()

我们之所以看到'多余'的'3dada,是因为什么呢, 是子生成器的StopIterration吗?'
1dada,对应每个boy
3dada,对应每次预激
2dada,对应每次预激结束


StopIteration: 

In [94]:
'''
书接上文, 我们在困惑为什么去掉while True时候就会爆出异常,
思考: 异常谁爆出来的呢, 我们知道没有while True的时候, 在委派生成器中, 由于子生成器在识别到None时执行了return, 这时候, 委派生成器就会
执行到yield from的左边, 把值(resclass元组)赋给字典, 那么委派生成器内部会继续向下执行, 这时候, 如果没有while true, 那代码就到了尽头
, 到了尽头自然爆出StopIteration异常, 该异常向上冒泡, 就显示为coroutine.send(None)这句代码出现异常.
那么如何在不用while True的情况下避免这个异常呢? 答案呼之欲出了, 自然是try, except代码块. 于是下面的代码就出现了, 完美解决了
while True的困惑, 并且避免了使用while True的情况下"3dada"重复打印的情况.
完美!
不过能不能用更好的方式去解决呢, 或者说, 能不能把这种try/except的逻辑写成一段固定的代码, 用更简洁的语法来实现...可能已经有了吧.
'''
from collections import namedtuple

ResClass = namedtuple('Res', 'count, average')

def averager():
    total = 0.0
    count = 0
    average = None
    
    while True:
        if count == 0:
            print("3dada,对应每次预激")
        term = yield  # 第一次预激是到这里暂停的.
        
        if term is None:
            break
        total += term
        count += 1
        average = total / count

    return ResClass(count, average)


def grouper(storages, key):
    storages[key] = yield from averager()


def client():
    process_data = {
        'boys_1': [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
        'boys_2': [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46]
    }

    storages = {}
    
    for k, v in process_data.items():
        coroutine = grouper(storages, k)
        
        print('1dada,对应每个boy')
        next(coroutine)
        print("2dada,对应每次预激结束")
        
        for dt in v:
            coroutine.send(dt)

        try:
            coroutine.send(None)
        except StopIteration:
            print("我不会让异常出现的")
            pass
    print(storages)

client()
print("完美!")

1dada,对应每个boy
3dada,对应每次预激
2dada,对应每次预激结束
我不会让异常出现的
1dada,对应每个boy
3dada,对应每次预激
2dada,对应每次预激结束
我不会让异常出现的
{'boys_1': Res(count=9, average=40.422222222222224), 'boys_2': Res(count=9, average=1.3888888888888888)}
完美!


# 还剩下的内容
- xml, json
- re, xpath
- 网络编程： socket, ftp, mail
- http协议, ==http web server 小项目
- django, 详细
- 爬虫, API

# asyncio (async I/O)原意:asynchronous IO, 即异步IO
- python3.4开始引入标准库当中, 内置对异步IO的支持
- asyncio本身是一个消息循环
- 步骤:
    - 创建消息循环
    - 把携程导入
    - 关闭

In [11]:
help(asyncio.get_event_loop)

Help on function get_event_loop in module asyncio.events:

get_event_loop()
    Return an asyncio event loop.
    
    When called from a coroutine or a callback (e.g. scheduled with call_soon
    or similar API), this function will always return the running event loop.
    
    If there is no running event loop set, the function will return
    the result of `get_event_loop_policy().get_event_loop()` call.



In [10]:
import threading
# 引入异步IO包
import asyncio

# 使用协程
@asyncio.coroutine
def hello():
    print('Hello world! (%s)' % threading.currentThread())
    print('Start....... (%s)' % threading.currentThread())
    yield from asyncio.sleep(3)
    print('Done........ (%s)' % threading.currentThread())
    print('Hello again! (%s)' % threading.currentThread())

# 启动消息循环    
loop = asyncio.get_event_loop()
# 定义任务
tasks = [
    asyncio.ensure_future(hello()), asyncio.ensure_future(hello())
        ]
                                                            
# asyncio使用wait等待task执行完毕
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

# 协程本质上是在单线程内部运行的, 它是代码的一种运行规则.

RuntimeError: This event loop is already running

Hello world! (<_MainThread(MainThread, started 140626472912704)>)
Start....... (<_MainThread(MainThread, started 140626472912704)>)
Hello world! (<_MainThread(MainThread, started 140626472912704)>)
Start....... (<_MainThread(MainThread, started 140626472912704)>)
Done........ (<_MainThread(MainThread, started 140626472912704)>)
Hello again! (<_MainThread(MainThread, started 140626472912704)>)
Done........ (<_MainThread(MainThread, started 140626472912704)>)
Hello again! (<_MainThread(MainThread, started 140626472912704)>)


# B站教程
1. 协程不是计算机中真实存在的, 是程序员认为创造, 也叫微线程, 单线程内用户态上下文切换

- 实现协程方法
    - greenlet, 早期模块
    - yield生成器, 关键字, 伪造, 没什么意义?
    - asyncio, Pyhton3.4之后
    - async await, Python3.5之后__推荐__

In [6]:
from greenlet import greenlet

def func1():
    print(11)
    gr2.switch()
    print(12)
    gr2.switch()
    
def func2():
    print(21)
    gr1.switch()
    print(22)
    
gr1 = greenlet(func1)
gr2 = greenlet(func2)

gr1.switch()  # start from func1

11
21
12
22


## asyncio
- python3.4之后

In [15]:
# 遇到IO阻塞自动切换
import asyncio

@asyncio.coroutine
def func1():
    print(11)
    yield from asyncio.sleep(0.5)
    print(12)

@asyncio.coroutine
def func2():
    print(21)
    yield from asyncio.sleep(.5)
    print(22)
     
        
tasks = [
    asyncio.ensure_future(func1()),
    asyncio.ensure_future(func2())
]
                                                            
loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

RuntimeError: This event loop is already running

11
21
12
22


## async & await 关键字
- python3.5之后

### 协程的意义
- 在一个线程中遇到IO等待,线程不会傻等, 利用空闲时间处理其他任务
- 顺序执行称之为'同步', 穿插执行称为'异步'.

In [31]:
# 案例:网络IO

# import requests

# def download_image(url):
#     print("开始下载", url)
#     # 发送网络请求, 下载图片
#     response = requests.get(url)
#     print("搞定")
#     # 图片保存到本地
#     file_name = url.rsplit('_')[36:]
#     with open(file_name, mode='wb') as fb:
#         fb.write(response.content)
   
# if __name__ == '__main__':
#     url_list = [
#         'http://wallpaperswide.com/download/silver_satin-wallpaper-1920x1080.jpg',
#         'http://wallpaperswide.com/download/sexy_brunette_3-wallpaper-1920x1080.jpg',
#         'http://wallpaperswide.com/download/red_lipstick-wallpaper-1920x1080.jpg',
#         'http://wallpaperswide.com/download/beautiful_girl_retro_artwork-wallpaper-1920x1080.jpg',
#         'http://wallpaperswide.com/download/kim_ha_yul-wallpaper-1920x1080.jpg'
#     ]


#     for item in url_list:
#         download_image(item)


## 异步编程
- 事件循环
    - task
        - task_list= \[asyncio.create_task(func()), \]
        - await asyncio.wait(task)
    - asyncio.Future

### concurrent.futures.Future对象
- 进程池/线程池实现异步操作时用到的对象
- 两个Future可能会需要交叉使用

### 异步迭代器

# async and await
- 为了更好的表示异步IO
- 让协程代码更简洁
- 使用上，可以进行简单的替换
    - 用async替换@asyncio.coroutine
    - await 替换 yield from
    

# aiohttp
- asyncio实现单线程的并发IO，在客户端用处不大
- 在服务器端可以asyncio+coroutine配合， 因为http是IO操作
- asyncio实现了tcp, udp, ssl等协议
- aiohttp是基于asyncio实现的http框架
- pip install aiohttp

In [ ]:
# aiohttp案例

# concurrent.futures
- python3新增的库
- 类似其他语言的线程池的概念
- 利用multiprocession实现真正的并行计算
- 核心原理：以子进程的形式， 并行运算多个python解释器
- concurrent.futures.Executor
    - ThreadPoolExecutor
    - ProcessPoolExecutor
    - 执行的时候需要自行选择
- submit(fn, args, kwargs)
    - fn 异步执行的参数
    - args, kwargs
    
## Future
    - 

## current map
- map(fn, iterable, timeout=None)
    - 函数需要异步执行